# Value Investing – Fair and Intrinsic Value Calculator (Beginner Friendly)


This notebook helps you:
- upload a CSV of fundamental data
- calculate intrinsic value using:
- Discounted Cash Flow (DCF)
- PER multiple
- Graham Number
- calculate Margin of Safety (MoS)
- export results to CSV


⚠️ Educational purpose only. Not an investment advice.

In [ ]:
# Clone github repository for external function
!git clone https://github.com/nuralibasyah/intrinsic-value-calculator.git
%cd intrinsic-value-calculator

In [ ]:
# Install dependencies
!pip install yfinance pandas numpy
import numpy as np
import pandas as pd
from datetime import datetime
from src.fetch_price import get_market_price
from src.valuation import calc_dcf_iv, calc_per_iv, calc_pbv_iv, calc_graham_iv

In [ ]:
# Upload your fundamental data here.
# Template are available at https://github.com/nuralibasyah/intrinsic-value-calculator/blob/main/data/template.csv
from google.colab import files
uploaded = files.upload()

In [ ]:
csv_path = list(uploaded.keys())[0]
df = pd.read_csv(csv_path)

# Make sure fundamental data are uploaded correctly
df

In [ ]:
# Set your parameter based on assumptions
DCF_GROWTH_RATE = 0.08
DCF_DISCOUNT_RATE = 0.12
DCF_TERMINAL_GROWTH = 0.02
DCF_YEARS = 5

PER_FAIR = 10
PBV_FAIR = 1.5

In [ ]:
# Appending the dataframe with valuation results
df["dcf_value"] = df.apply(lambda r: calc_dcf_iv(r, DCF_GROWTH_RATE, DCF_DISCOUNT_RATE, DCF_TERMINAL_GROWTH, DCF_YEARS), axis=1)
df["per_value"] = df.apply(lambda r: calc_per_iv(r, PER_FAIR), axis=1)
df["pbv_value"] = df.apply(lambda r: calc_pbv_iv(r, PBV_FAIR), axis=1)
df["graham_value"] = df.apply(calc_graham_iv, axis=1)

# Combining 4 valuation methods, you can change the weight of each methods as long as they sums up to 1
df["combined_value"] = 0.25 * df["dcf_value"] + 0.25 * df["per_value"] + 0.25 * df["graham_value"] + 0.25 * df["pbv_value"]

# Get current market price using yfinance
df["market_price"] = df.apply(get_market_price, axis=1)

# Calculate margin of safety and categorize the result (mos < 0 = overvalued, mos > 0.3 undervalued, else = fair)
df["margin_of_safety"] = (df["combined_value"] - df["market_price"]) / df["market_price"]
df["valuation"] = np.select([df["margin_of_safety"] < 0, df["margin_of_safety"] > 0.3],["overvalued","undervalued"], default="fair")
df

In [ ]:
# Download valuation result to csv with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"Intrinsic_Valuation_Result_{timestamp}.csv"

df.to_csv(filename, index=False)
files.download(filename)